In [1]:
import os
os.chdir('../')
%pwd

'/home/reby/Desktop/personal_Projects/car_price_prediction'

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rby756/used_car_pred.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rby756"
os.environ["MLFLOW_TRACKING_PASSWORD"]="Goldy@7329"

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.carPricePrediction.constants import *
from src.carPricePrediction.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri=config.mlflow_uri,  
        )

        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, y_test, y_pred):
        
        # Evaluate the model
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)

        return mae,mse,rmse,r2
    

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():

            predictions = model.predict(test_x)

            (mae,mse,rmse,r2) = self.eval_metrics(test_y, predictions)
            
            # Saving metrics as local
            scores = {"mae": mae, "mse": mse, "rmse": rmse, "r2_score": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("mae",mae)
            mlflow.log_metric("mse",mse)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2_score", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestRegressor")
            else:
                mlflow.sklearn.log_model(model, "model")

In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-09 18:04:17,857: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-09 18:04:17,860: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 18:04:17,865: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 18:04:17,867: INFO: common: created directory at: artifacts]
[2024-07-09 18:04:17,868: INFO: common: created directory at: artifacts/model_evaluation]
[2024-07-09 18:04:18,571: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/home/reby/Desktop/personal_Projects/car_price_prediction/venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
Successfully registered model 'RandomForestRegressor'.
2024/07/09 18:06:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressor, version 1
Created version '1' of model 'RandomForestRegressor'.
